# Deduplicated public buyers

In [1]:
import pandas as pd
import numpy as np

In [2]:
CNTS = '/home/rdora/declaranet/data/pre-process/contratos_3.csv'
cnts = pd.read_csv(CNTS)

In [21]:
public = cnts.groupby(['buyer', 'buyer_state', 'ramo', 'gvmnt_level'])[
    ['buyer', 'buyer_state', 'ramo', 'gvmnt_level']].head()

In [23]:
PUBLIC = '/home/rdora/declaranet/data/dedup/public.csv'
public.to_csv(PUBLIC, index=False)

In [26]:
public.buyer_state.unique()

array(['Sinaloa', 'México', 'Chiapas', 'Baja California Sur', 'Morelos',
       'Guanajuato', 'Colima', 'Veracruz de Ignacio de la Llave',
       'Ciudad de México', 'Sonora', 'Oaxaca', 'Hidalgo', 'Nayarit',
       'Tamaulipas', 'Baja California', 'Yucatán', 'Nuevo León',
       'Durango', 'Coahuila de Zaragoza', 'Jalisco',
       'Michoacán de Ocampo', 'Querétaro', 'San Luis Potosí',
       'Quintana Roo', 'Guerrero', 'Zacatecas', 'Puebla', 'Campeche',
       'Tabasco', nan, 'Aguascalientes', 'Chihuahua', 'Tlaxcala'],
      dtype=object)

# Run Dedup

Here, we run the script `~/declaranet/python/deduplicate.py` to create a cluster for each record.

In [63]:
PUBLIC_2 = '/home/rdora/declaranet/data/dedup/output_public.csv'
public_cl = pd.read_csv(PUBLIC_2)

# Groupby

In [16]:
def get_mode(group):
    vc = group.value_counts()
    if vc.shape[0]:
        return vc.index[0]
    else:
        return np.nan

In [64]:
gb_public = public_cl.groupby('Cluster ID')['buyer', 'buyer_state'].agg(get_mode).reset_index()

/home/rdora/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


## Companies

In [66]:
public = cnts.copy()

public = pd.merge(public,
                public_cl[['Cluster ID', 'buyer', 'buyer_state']],
                on=['buyer', 'buyer_state'],
                how='left')

public = public.drop(['buyer', 'buyer_state'], axis=1)

public = pd.merge(public, gb_public, how='left')

In [67]:
print("New contracts with Buyer State:",
      cnts.buyer_state.isna().sum() - public.buyer_state.isna().sum())

print("Number of duplicated UCs:",
     cnts.buyer.nunique() - public.buyer.nunique())

New contracts with Buyer State: 0
Number of duplicated UCs: 178


In [68]:
buyer = set(cnts.buyer.unique())
buyer_2 = set(public.buyer.unique())
miss = buyer - buyer_2

print(f"Number of deduplicated edges: {cnts[cnts.buyer.isin(miss)].shape[0]:,}")

Number of deduplicated edges: 88,102
